In [1]:

import datetime


import time

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from LTSF_LinearModel_lib import LTSF_DLinear, LTSF_NLinear
from sklearn.preprocessing import MinMaxScaler

# import plotly
# import plotly.graph_objects as go
# import plotly_express as px
# 调用BTC爬取部分
import sys
sys.path.append("/mnt/量化交易/")  # 增加指定的绝对路径,进入系统路径,从而便于该目录下的库调用

from BTCCrawl_To_DataFrame_Class import BTC_data_acquire as BTC

Folder_base = "/mnt/量化交易/data/"
URL = "https://api.coincap.io/v2/candles?exchange=huobi&interval=h12&baseId=bitcoin&quoteId=tether"
StartDate = "2022-09-10"
EndDate = "2022-12-01"
BTC_json = "BTC_h12.json"

In [2]:
BTC_data = BTC(URL, StartDate, EndDate, Folder_base, BTC_json)
data0 = BTC_data.MarketFactor_DataFrame(
    FromWeb=False,
    Market_Factor=True,
    weekdays=7,
)
# data0["year"] = (
#     data0["year"] - 2016
# )  # 因为数据从2016年开始,减去2016,将年份变成0以上的整数,以使得Embedding词汇表范围缩小


def dataset_Scaler(data):
    # 将原始数据MinMax归一化,以提高特征辨识度; 注意:要小心,重复归一化,可能会导致数据变小
    # 将close_price单独归一化,因为后面需要将close_price预测出后,还原
    X_Scaler = MinMaxScaler()
    y_Scaler = MinMaxScaler()

    X_ds = X_Scaler.fit_transform(
            data[data.columns[:-1]]
        )  # 本是DataFrame的类型,经过转换,X变成了ndarray类型;
    y_ds = y_Scaler.fit_transform(
        data["close"].to_numpy().reshape(-1, 1)
    )  # y为series,标准化需要(-1,1)的shape
    data = np.c_[X_ds, y_ds]

    return data, X_Scaler, y_Scaler


data, X_Scaler, y_Scaler = dataset_Scaler(data0)
print(type(data))
print("data.shape={}".format(data.shape))
print("最后两行数据:\n{}".format(data[-2:, :]))

爬取数据DataFrame.shape
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4349 entries, 2016-05-29 12:00:00 to 2022-10-15 00:00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    4349 non-null   float32
 1   high    4349 non-null   float32
 2   low     4349 non-null   float32
 3   close   4349 non-null   float32
 4   volume  4349 non-null   float32
dtypes: float32(5)
memory usage: 118.9 KB
从web中读出Data.infor:None
BTC数据合并存入:/mnt/量化交易/data/BTC_h12.json


,d_return,d_amplitude,volume,RSI_7,Log_close_weeklag,Log_high_low,Log_open_weeklag,open_pre_close,high_pre_close,low_pre_close,high,low,open,close
2016-06-05 12:00:00,0.002604,13.5,108.0,76.923077,0.081125,0.023408,0.081696,1.006944,1.013021,0.989583,583.5,570.0,580.0,577.5
2016-06-06 00:00:00,0.009524,10.0,81.0,78.244701,0.082189,0.017272,0.080259,0.999134,1.011255,0.993939,584.0,574.0,577.0,583.0


,d_return,d_amplitude,volume,RSI_7,Log_close_weeklag,Log_high_low,Log_open_weeklag,open_pre_close,high_pre_close,low_pre_close,high,low,open,close
2022-10-14 12:00:00,-0.021340,770.990234,2840.872314,46.267878,0.001312,0.039623,0.024056,1.000000,1.012755,0.973412,19846.349609,19075.359375,19596.390625,19178.210938
2022-10-15 00:00:00,-0.001835,87.988281,438.222687,45.725215,0.004294,0.004588,0.001185,0.999874,1.002364,0.997776,19223.539062,19135.550781,19175.789062,19143.019531


<class 'numpy.ndarray'>
data.shape=(4335, 14)
最后两行数据:
[[0.40962179 0.07436248 0.10554655 0.43683048 0.54989643 0.09857616
  0.57751997 0.28983045 0.30083152 0.45864359 0.28179938 0.27919698
  0.28434607 0.27813715]
 [0.44684017 0.00848652 0.01626705 0.43023403 0.55325276 0.01141336
  0.5514983  0.2895598  0.27816732 0.5014807  0.27269928 0.28009989
  0.27807294 0.27761221]]


In [3]:
# best hyperparameter-----
Batch_size = 64
kernel_size = 57
L_seq = 208
individual = (False,)
L_pred = 7
out_features = 1


def gen(data):
    for i in range(data.shape[0]):
        yield data[i, :]


def xs_dataset(
    data,
    L_seq,
    L_pred,
    Batch_size,
    out_features=1,
    shuffle=False,
    buffer_size=10000,
):
    """
    args:
        data: Dataframe格式的数据(B,in_features);
        L_seq: Linear模型输入的时序长度;
        L_pred: Linear模型输出的预测时序长度;
        out_features: target输出时的特征数,缺省=1,只取收盘价格
        Batch_size:
        shuffle: bool ; 是否shuffle
    out:
        (xs,ys): (input, target) ((B,L_seq,features),(B,L_pred,out_features))

    """
    output_signature = tf.TensorSpec((14), tf.float64)  # 14列
    # gen_f = gen(data)
    x_y = tf.data.Dataset.from_generator(
        gen,
        output_signature=output_signature,
        args=(data,),
    )  # args用于给gen传递参数,必须是元组的形式传入参数;
    # x_y = tf.data.Dataset.from_tensor_slices(data)
    x_y = x_y.window(L_seq + L_pred, shift=1, drop_remainder=True)
    x_y = x_y.flat_map(lambda w: w.batch(L_seq + L_pred, drop_remainder=True))
    xs = x_y.map(lambda w: w[:L_seq, :])  # 时序L_seq;in_features中,不包含时间戳部分;
    ys = x_y.map(
        lambda w: w[L_seq:, (out_features * -1) :]
    )  # 时序L_pred部分;in_features中,不包含时间戳部分;作为Target标签,这里只取最后的收盘价格,shape(B,L,1)

    dataset = tf.data.Dataset.zip((xs, ys))
    if shuffle == True:
        dataset.shuffle(buffer_size)

    return dataset.batch(Batch_size, drop_remainder=True).prefetch(1)


split = int(data.shape[0] * 0.8)
# split = 2 # 为调试使用,缩短运行时间
print("data共有样本:{};train/val分隔split:{}".format(data.shape[0], split))

dataset_train = xs_dataset(
    data[:split, :],
    L_seq,
    L_pred,
    Batch_size,
    out_features,
    shuffle=True,
    buffer_size=10000,
)
print("dataset_train元组长度:{}".format(len(iter(dataset_train).next())))
print("dataset_train中xs.shape:{}".format(iter(dataset_train).next()[0].shape))
print("dataset_train中ys.shape:{}".format(iter(dataset_train).next()[1].shape))

dataset_val = xs_dataset(
    data[split:, :],
    L_seq,
    L_pred,
    Batch_size,
    shuffle=True,
    buffer_size=10000,
)
print("dataset_val元组长度:{}".format(len(iter(dataset_val).next())))
print("dataset_val中xs.shape:{}".format(iter(dataset_val).next()[0].shape))
print("dataset_val中ys.shape:{}".format(iter(dataset_val).next()[1].shape))

data共有样本:4335;train/val分隔split:3468


2022-10-15 07:41:12.008804: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-15 07:41:12.574758: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10801 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0000:11:00.0, compute capability: 3.7
2022-10-15 07:41:13.021078: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


dataset_train元组长度:2
dataset_train中xs.shape:(64, 208, 14)
dataset_train中ys.shape:(64, 7, 1)
dataset_val元组长度:2
dataset_val中xs.shape:(64, 208, 14)
dataset_val中ys.shape:(64, 7, 1)


In [4]:
# 初始化模型:
DLinear = LTSF_DLinear(
    seq_len=L_seq,
    pred_len=L_pred,
    channels=14,
    kernel_size=25,
    individual=True,
)

# 1) 自定义loss,metrics:
#  loss:
loss_object = tf.keras.losses.MeanAbsoluteError(
    reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE
)  # 求和

#  metric
train_loss = tf.keras.metrics.Mean(name="train_loss")
val_loss = tf.keras.metrics.Mean(name="val_loss")
train_MSE = tf.keras.metrics.MeanSquaredError(name="train_MSE")
val_MSE = tf.keras.metrics.MeanSquaredError(name="val_MSE")

# 2) 自定义learning_rate,从而自定义optimizer:
# 自定义 learning_rate
def dict_compare(
    initial, lr_dict, step
):  # 因为If等python语句不能在Tensorflow中转化成Graph,这里用py_function包装
    """
    用于从lr列表中,提取与训练step相对应的lr
    """
    lr = initial
    index = tf.where(lr_dict[:, 0] <= step)  # 找到最接近step的index
    if index.numpy().size != 0:
        lr = lr_dict[:, 1].numpy()[max(index).numpy()]  # ndary shape:(1,)
        lr = lr[0]
    return lr


class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):  # 定义Schedule类
    def __init__(self, initial):  # 不能添加**kwargs;与自定义layer层不一样;
        """
        args:
            initial: learning_rate initial value, 5e-3 for example
        """
        super(CustomSchedule, self).__init__()
        self.initial = initial
        self.lr_dict = tf.convert_to_tensor(
            (
                (2, initial),
                (4, initial / 5),
                (6, initial / 10),
                (8, initial / 50),
                (10, initial / 100),
                (15, initial / 500),
                (20, initial / 1000),
                #                 (80, initial / 5000),
            )
        )

    def __call__(self, step):
        lr = tf.py_function(
            dict_compare, inp=[self.initial, self.lr_dict, step], Tout=tf.float32
        )  # py_function包装python函数
        return lr


lr = CustomSchedule(5e-03)  # build lr类;

# for step in [2,4,6,8,10,15,20]:
#     print(lr(step),end=',')

# optimizer:
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# 3) 自定义Callbacks:


# 非Keras方法,直接设立checkpoing;
checkpoint_path = "./checkpoint"
ckpt = tf.train.Checkpoint(DLinear=DLinear, optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(
    ckpt,
    checkpoint_path,
    max_to_keep=1,
    checkpoint_name="BTC_221014",
)


callbacks = tf.keras.callbacks.CallbackList(
    callbacks=[], add_history=True, add_progbar=True, model=DLinear
)  # callbacks=[]初始为空,即在训练时,出现进度条;其它callbacks使用append方法添加;
# customcallback = CustomCallback()
# callbacks.append(customcallback) #测试成功,没有实际意义,关闭;
# callbacks.append(Modelcheckpoint) #keras与custom training在这点,兼容性有问题;建议采用tf.train.checkpoint方法;
# callbacks.append(Earlystop) #keras与custom training兼容性不方便,建议直接在cutsom traning中代码实现;


# 3) 自定义单步训练:
@tf.function  # 该 @tf.function 将追踪-编译 train_step 到 TF 图中，以便更快地执行。
def train_step(dataset):  # dataset是xs与xs_timestamp合集
    xs, ys = dataset[0], dataset[1]

    # 求导,根据导数优化变量
    with tf.GradientTape() as tape:
        y_ = DLinear(xs)
        loss_value = loss_object(ys, y_[:,:,-1:])
    gradients = tape.gradient(loss_value, DLinear.trainable_variables)
    optimizer.apply_gradients(zip(gradients, DLinear.trainable_variables))
    # 计算metric: 每个单步经过batch次运算后,共batch次 loss的平均值
    train_loss.update_state(loss_value)
    train_MSE.update_state(ys, y_[:,:,-1:])
    return {"train_loss": train_loss.result(), "train_MSE": train_MSE.result()}


@tf.function
def test_step(dataset_val):  # validation, test 使用该单步训练
    xs, ys = dataset_val[0], dataset_val[1]
    y_ = DLinear(xs) #输出:(Batch,L_pred,channels)
    loss_value = loss_object(ys, y_[:,:,-1:])
    val_loss.update_state(loss_value)
    val_MSE.update_state(ys, y_[:,:,-1:])
    return {"val_loss": val_loss.result(), "val_MSE": val_MSE.result()}


def Modelevaluation(dataset_val, training=False):  # 训练后的模型Evaluation,并输出target预测值
    val_loss.reset_states()
    val_MSE.reset_states()
    target_spec = dataset_val.element_spec[1]
    y_hat = tf.zeros(target_spec.shape, target_spec.dtype)
    for batch_data in dataset_val:
        xs, ys = batch_data[0], batch_data[1]
        y_ = DLinear(xs)
        loss_value = loss_object(ys, y_[:,:,-1:])
        val_loss.update_state(loss_value)
        val_MSE.update_state(ys, y_[:,:,-1:])
        # y_hat = tf.stack([y_hat,y_.numpy()])
        y_hat = tf.concat([y_hat, y_.numpy()[:,:,-1:]], axis=0)
    print(
        "training={},val_loss:{:.4f},val_MSE:{:.4f}".format(
            training, val_loss.result().numpy(), val_MSE.result().numpy()
        )
    )
    return y_hat

In [5]:
# 4) epoch循环训练:
EPOCHS = 1000
train_loss_results = []
logs = {}  # 字典
wait = 0
best = np.infty  # 先设置一个无穷大的数字;
patience = 15

train_start = time.time()
callbacks.on_train_begin(logs=logs)
for epoch in range(EPOCHS):
    epoch_start = time.time()
    callbacks.on_epoch_begin(epoch, logs=logs)

    train_loss.reset_states()
    train_MSE.reset_states()

    for batch, batch_data in enumerate(dataset_train):
        callbacks.on_batch_begin(batch, logs=logs)
        callbacks.on_train_batch_begin(batch, logs=logs)
        train_dict = train_step(batch_data)
        logs["train_loss"] = train_dict["train_loss"]
        logs["train_MSE"] = train_dict["train_MSE"]

        # if batch % 10 == 0:  # 每n次batch,打印
        #     print(
        #         "Epoch {} Batch {} train_loss {:.4f} train_MSE {:.4f} ".format(
        #             epoch + 1, batch, train_loss.result(), train_MSE.result()
        #         )
        #     )
        callbacks.on_train_batch_end(batch, logs=logs)
        callbacks.on_batch_end(batch, logs=logs)
    # End epoch 每个epoch
    train_loss_results.append(train_loss.result())

    val_loss.reset_states()
    val_MSE.reset_states()
    for batch_data in dataset_val:
        callbacks.on_batch_begin(batch, logs=logs)
        callbacks.on_test_batch_begin(batch, logs=logs)
        test_step(batch_data)
        callbacks.on_test_batch_end(batch, logs=logs)
        callbacks.on_batch_end(batch, logs=logs)

    logs["val_loss"] = val_loss.result()
    logs["val_MSE"] = val_MSE.result()
    # --------------------
    # The early stopping strategy: stop the training if `val_loss` does not
    # decrease over a certain number of epochs.
    wait += 1
    if (
        val_loss.result() < best
    ):  # 当loss变小,在改进时,计数器恢复为0,存储模型,实现总是存储最佳模型; 当n次loss不再变小,即触发
        best = val_loss.result()
        wait = 0

        ckpt_save_path = ckpt_manager.save()  # 存weight
        # print("Saving checkpoint for epoch {} at {}".format(epoch + 1, ckpt_save_path))
    if wait >= patience:
        print(
            "\nepoch:{}/{} - 共耗时:{:.2f}分,历{}次训练未见val_loss减少,故提前中止".format(
                epoch + 1, EPOCHS, (time.time() - train_start) / 60, patience
            )
        )
        break
    # --------------------

    if (epoch + 1) % 10 == 0:  # 每n次epoch,打印

        print(
            "\nepoch:{}/{} - 耗时:{:.2f}分/总{:.2f}分; train_loss {:.4f} train_MSE {:.4f}; val_loss {:.4f} val_MSE {:.4f}".format(
                epoch + 1,
                EPOCHS,
                (time.time() - epoch_start) / 60,
                (time.time() - train_start) / 60,
                train_loss.result(),
                train_MSE.result(),
                val_loss.result(),
                val_MSE.result(),
            )
        )
    # print("Time taken for 1 epoch: {} mins\n".format((time.time() - start) / 60))
    callbacks.on_epoch_end(epoch, logs=logs)
callbacks.on_train_end(logs=logs)
print(
    "\nepoch:{}/{} - 共耗时:{:.2f}分,正常结束".format(
        epoch + 1, EPOCHS, (time.time() - train_start) / 60
    )
)

# 在Docker上运行时,用于JupyterLab上自动关闭租用连接:
# import os
# 若释放前要保存环境并命名为 SnapName
# os.system("export $(cat /proc/1/environ |tr '\\0' '\\n' | grep MATCLOUD_CANCELTOKEN)&&/public/script/matncli node cancel -url https://matpool.com/api/public/node -save -name snapName")
# 若释放前不需要保存环境
# os.system("export $(cat /proc/1/environ |tr '\\0' '\\n' | grep MATCLOUD_CANCELTOKEN)&&/public/script/matncli node cancel -url https://matpool.com/api/public/node")

2022-10-15 07:41:37.335741: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101


50/50 [==============================] - ETA: 0s - train_loss: 0.0594 - train_MSE: 0.0135 - val_loss: 0.5618 - val_MSE: 0.5171
epoch:10/1000 - 耗时:0.04分/总0.50分; train_loss 0.1184 train_MSE 0.0542; val_loss 0.5526 val_MSE 0.4998
50/50 [==============================] - ETA: 0s - train_loss: 0.0518 - train_MSE: 0.0097 - val_loss: 0.4763 - val_MSE: 0.3640
epoch:20/1000 - 耗时:0.04分/总0.95分; train_loss 0.1017 train_MSE 0.0391; val_loss 0.4684 val_MSE 0.3509
50/50 [==============================] - ETA: 0s - train_loss: 0.0458 - train_MSE: 0.0071 - val_loss: 0.4027 - val_MSE: 0.2498
epoch:30/1000 - 耗时:0.04分/总1.40分; train_loss 0.0879 train_MSE 0.0282; val_loss 0.3960 val_MSE 0.2404
50/50 [==============================] - ETA: 0s - train_loss: 0.0412 - train_MSE: 0.0054 - val_loss: 0.3401 - val_MSE: 0.1688
epoch:40/1000 - 耗时:0.04分/总1.84分; train_loss 0.0768 train_MSE 0.0210; val_loss 0.3345 val_MSE 0.1623
50/50 [==============================] - ETA: 0s - train_loss: 0.0382 - train_MSE: 0.0044 - 

In [ ]:
# DLinear = LTSF_DLinear(
#     seq_len=L_seq,
#     pred_len=L_pred,
#     channels=14,
#     kernel_size=25,
#     individual=False,
# )
# DLinear(iter(dataset_train).next()[0])

In [ ]:
import keras_tuner as kt


# 定义超参空间Class
class HypermodelSpace(kt.HyperModel):
    def build(self, hp):
        # 定义超参选项
        hp_Batch_size = hp.Int("Batch_size", min_value=32, max_value=128, step=32)
        hp_kernel_size = hp.Int("kernel_size", min_value=24, max_value=60, step=1)
        hp_L_seq = hp.Int("L_seq", min_value=48, max_value=336, step=32)
        hp_individual = hp.Boolean("individual", default=True)

        model = LTSF_DLinear(
            seq_len=hp_L_seq,
            pred_len=L_pred,
            channels=14,
            kernel_size=hp_kernel_size,
            individual=hp_individual,
        )

        return model

    def fit(self, hp, model, *args, **kwargs):
        hp_Batch_size = hp.get("Batch_size")
        hp_L_seq = hp.get("L_seq")

        dataset_train = xs_dataset(
            data[:split, :],
            hp_L_seq,
            L_pred,
            hp_Batch_size,
            out_features,
            shuffle=True,
            buffer_size=10000,
        )

        dataset_val = xs_dataset(
            data[split:, :],
            hp_L_seq,
            L_pred,
            hp_Batch_size,
            out_features,
            shuffle=True,
            buffer_size=10000,
        )

        EPOCHS = 20
        train_loss_results = []
        logs = {}  # 字典
        wait = 0
        best = np.infty  # 先设置一个无穷大的数字;

        patience = 20

        # # Assign the model to the callbacks.
        # for callback in callbacks:
        #     callback.model = model
        # callbacks.set_model(model=model)
        callbacks = tf.keras.callbacks.CallbackList(
            callbacks=[], add_history=True, add_progbar=True, model=model
        )

        @tf.function  # 该 @tf.function 将追踪-编译 train_step 到 TF 图中，以便更快地执行。
        def train_step(dataset):  # dataset是xs与xs_timestamp合集
            xs, ys = dataset[0], dataset[1]

            # 求导,根据导数优化变量
            with tf.GradientTape() as tape:
                y_ = model(xs)
                loss_value = loss_object(ys, y_[:, :, -1:])
            gradients = tape.gradient(loss_value, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            # 计算metric: 每个单步经过batch次运算后,共batch次 loss的平均值
            train_loss.update_state(loss_value)
            train_MSE.update_state(ys, y_[:, :, -1:])
            return {"train_loss": train_loss.result(), "train_MSE": train_MSE.result()}

        @tf.function
        def test_step(dataset_val):  # validation, test 使用该单步训练
            xs, ys = dataset_val[0], dataset_val[1]
            y_ = model(xs)
            loss_value = loss_object(ys, y_[:, :, -1:])
            val_loss.update_state(loss_value)
            val_MSE.update_state(ys, y_[:, :, -1:])
            return {"val_loss": val_loss.result(), "val_MSE": val_MSE.result()}

        train_start = time.time()
        callbacks.on_train_begin(logs=logs)
        for epoch in range(EPOCHS):
            epoch_start = time.time()
            callbacks.on_epoch_begin(epoch, logs=logs)

            train_loss.reset_states()
            train_MSE.reset_states()

            for batch, batch_data in enumerate(dataset_train):
                callbacks.on_batch_begin(batch, logs=logs)
                callbacks.on_train_batch_begin(batch, logs=logs)
                train_dict = train_step(batch_data)
                logs["train_loss"] = train_dict["train_loss"]
                logs["train_MSE"] = train_dict["train_MSE"]

                # if batch % 10 == 0:  # 每n次batch,打印
                #     print(
                #         "Epoch {} Batch {} train_loss {:.4f} train_MSE {:.4f} ".format(
                #             epoch + 1, batch, train_loss.result(), train_MSE.result()
                #         )
                #     )
                callbacks.on_train_batch_end(batch, logs=logs)
                callbacks.on_batch_end(batch, logs=logs)
            # End epoch 每个epoch
            train_loss_results.append(train_loss.result())

            val_loss.reset_states()
            val_MSE.reset_states()
            for batch_data in dataset_val:
                callbacks.on_batch_begin(batch, logs=logs)
                callbacks.on_test_batch_begin(batch, logs=logs)
                test_step(batch_data)
                callbacks.on_test_batch_end(batch, logs=logs)
                callbacks.on_batch_end(batch, logs=logs)

            logs["val_loss"] = val_loss.result()
            logs["val_MSE"] = val_MSE.result()
            # --------------------
            # The early stopping strategy: stop the training if `val_loss` does not
            # decrease over a certain number of epochs.
            #             wait += 1
            #             if (
            #                 val_loss.result() < best
            #             ):  # 当loss变小,在改进时,计数器恢复为0,存储模型,实现总是存储最佳模型; 当n次loss不再变小,即触发
            #                 best = val_loss.result()
            #                 wait = 0

            #                 ckpt_save_path = ckpt_manager.save()  # 存weight
            #                 # print("Saving checkpoint for epoch {} at {}".format(epoch + 1, ckpt_save_path))
            #             if wait >= patience:
            #                 print(
            #                     "\nepoch:{}/{} - 共耗时:{:.2f}分,历{}次训练未见val_loss减少,故提前中止".format(
            #                         epoch + 1, EPOCHS, (time.time() - train_start) / 60, patience
            #                     )
            #                 )
            #                 break
            # --------------------

            if (epoch + 1) % 10 == 0:  # 每n次epoch,打印

                print(
                    "\nepoch:{}/{} - 耗时:{:.2f}分/总{:.2f}分; train_loss {:.4f} train_MSE {:.4f}; val_loss {:.4f} val_MSE {:.4f}".format(
                        epoch + 1,
                        EPOCHS,
                        (time.time() - epoch_start) / 60,
                        (time.time() - train_start) / 60,
                        train_loss.result(),
                        train_MSE.result(),
                        val_loss.result(),
                        val_MSE.result(),
                    )
                )
            # print("Time taken for 1 epoch: {} mins\n".format((time.time() - start) / 60))
            best = min(
                best, float(logs["val_loss"])
            )  # 取最小的val_loss,val_loss需要转换成float数,才能被tunner接受
            callbacks.on_epoch_end(epoch, logs=logs)
        callbacks.on_train_end(logs=logs)
        print(
            "\nepoch:{}/{} - 共耗时:{:.2f}分,正常结束".format(
                epoch + 1, EPOCHS, (time.time() - train_start) / 60
            )
        )

        return best

In [ ]:
tuner = kt.Hyperband(
    hypermodel=HypermodelSpace(),
    objective=kt.Objective("val_loss", "min"),
    max_epochs=20,
    directory="Tunner_result",
    project_name="custom_training",
    overwrite=False,
)

In [ ]:
# 运行超参数搜索

start = time.time()
tuner.search(
    # inputs_train,
    epochs=20,
    # validation_data= inputs_val,
    # callbacks=callbacks, #callbacks是个objects,而这里必须是可以copy.deepcopy. callbacklists对象不能深度复制
)
end = time.time()
print("共耗时{:.3f}分钟".format((end - start) / 60))

# 在Docker上运行时,用于JupyterLab上自动关闭租用连接:
# import os
# 若释放前要保存环境并命名为 SnapName
# os.system("export $(cat /proc/1/environ |tr '\\0' '\\n' | grep MATCLOUD_CANCELTOKEN)&&/public/script/matncli node cancel -url https://matpool.com/api/public/node -save -name snapName")
# 若释放前不需要保存环境
# os.system("export $(cat /proc/1/environ |tr '\\0' '\\n' | grep MATCLOUD_CANCELTOKEN)&&/public/script/matncli node cancel -url https://matpool.com/api/public/node")

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=7)
for i in range(7):
    print(best_hps[i].values)

tuner.results_summary()